## List avaliable amazon bedrock models

In [ ]:

"""
Lists the available Amazon Bedrock models.
"""
import logging
import json
import boto3


from botocore.exceptions import ClientError


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models

    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise


def main():
    """Entry point for the example. Uses the AWS SDK for Python (Boto3)
    to create an Amazon Bedrock client. Then lists the available Bedrock models
    in the region set in the callers profile and credentials.
    """

    bedrock_client = boto3.client(service_name="bedrock")

    fm_models = list_foundation_models(bedrock_client)
    for model in fm_models:
        print(f"Model: {model['modelName']}")
        print(json.dumps(model, indent=2))
        print("---------------------------\n")

    logger.info("Done.")


if __name__ == "__main__":
    main()

## AWS sample code

## langchain_aws

In [12]:
# Use the Conversation API to send a text message to Amazon Nova.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Amazon Nova Lite.
model_id = "amazon.nova-lite-v1:0"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




The purpose of a 'hello world' program is to provide a simple introduction to the syntax and basic structure of a programming language.


## S3

### S3 - upload file & folder

In [ ]:
import boto3

s3_client = boto3.client('s3', 
                  region_name='us-east-1')

def upload_file_to_s3(bucket_name, file_path, object_name):
    try:
        s3_client.upload_file(file_path, bucket_name, object_name)
        print(f"File '{file_path}' uploaded to bucket '{bucket_name}' as '{object_name}'.")
    except Exception as e:
        print(f"Error uploading file: {e}")

# upload folder to s3
def upload_folder_to_s3(bucket_name, folder_path):
    import os
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            object_name = os.path.relpath(file_path, folder_path)
            upload_file_to_s3(bucket_name, file_path, object_name)

upload_folder_to_s3('aiwave-hackathon-team', '../data')

### S3 - get information

In [ ]:
import boto3

s3_client = boto3.client('s3', 
                  region_name='us-east-1')

def list_buckets():
    response = s3_client.list_buckets()
    buckets = response['Buckets']
    [print(f"Bucket Name: {bucket['Name']}") for bucket in buckets]
    return buckets

def get_objects(bucket_name, prefix=""):
    response = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=prefix  # 可選：只列出特定開頭的
    )
    print(f"Objects in bucket '{bucket_name}':")
    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
        return response['Contents']
    else:
        print("No objects found.")
        return []
        
# get s3 bucket-name arn
def get_s3_bucket_arn(bucket_name):
    return f"arn:aws:s3:::{bucket_name}"
# get s3 object arn
def get_s3_object_arn(bucket_name, object_name):
    return f"arn:aws:s3:::{bucket_name}/{object_name}"
# get s3 object url
def get_s3_object_url(bucket_name, object_name):
    return f"https://{bucket_name}.s3.amazonaws.com/{object_name}"

def get_s3_bucket_info(bucket_name):
    bucket_arn = get_s3_bucket_arn(bucket_name)
    objects = get_objects(bucket_name)
    object_arns = [get_s3_object_arn(bucket_name, obj['Key']) for obj in objects]
    object_urls = [get_s3_object_url(bucket_name, obj['Key']) for obj in objects]
    return {
        'bucket_name': bucket_name,
        'bucket_arn': bucket_arn,
        'objects': objects,
        'object_arns': object_arns,
        'object_urls': object_urls
    }


# get all buckets info
s3_bucket_info = get_s3_bucket_info("aiwave-hackathon-team")
print(f"Bucket Name: {s3_bucket_info['bucket_name']}")
print(f"Bucket ARN: {s3_bucket_info['bucket_arn']}")
print(f"Objects: {s3_bucket_info['objects']}")
print(f"Object ARNs: {s3_bucket_info['object_arns']}")
print(f"Object URLs: {s3_bucket_info['object_urls']}")
print("---------------------------\n")

## OpenSearch

In [ ]:
import boto3
REGION = 'us-east-1'  # 你的 AWS region
COLLECTION_NAME = 'rag-database'
aoss = boto3.client('opensearchserverless', region_name=REGION)

In [ ]:
# Step 1: List collections to find the ID
list_response = aoss.list_collections()
collection_id = None
collection_arn = None
for col in list_response['collectionSummaries']:
    if col['name'] == COLLECTION_NAME:
        collection_id = col['id']
        collection_arn = col['arn']
        break

print(f"Found Collection ID: {collection_id}")
print(f"Found Collection ARN: {collection_arn}")

## Bedrock Knowledge base

In [ ]:
KB_NAME = 'stainlness'
S3_BUCKET_NAME = 'aiwave-hackathon-team-8'
S3_BUCKET_ARN = f'arn:aws:s3:::{S3_BUCKET_NAME}'
S3_PREFIX = 'data/'  # S3 中的資料夾路徑
BEDROCK_USER_ARN = 'arn:aws:iam::904375567622:user/stainlness-rag'
BEDROCK_ROLE_ARN = 'arn:aws:iam::904375567622:role/bedrock_role'
EMBEDDING_MODEL_ARN = f'arn:aws:bedrock:{REGION}::foundation-model/amazon.titan-embed-g1-text-02'
OPENSEARCH_COLLECTION_ARN = collection_arn
OPENSEARCH_COLLECTION_ID = collection_id
OPENSEARCH_VECTOR_INDEX_NAME = 'text_vectors'
S3_BUCKET_ARN = s3_bucket_info['bucket_arn']
S3_BUCKET_PREFIX = ''  # 如果是整個 bucket 就留空 ""

In [ ]:
# 建立 Knowledge Base
bedrock_agent = boto3.client('bedrock-agent', region_name=REGION)
create_kb_response = bedrock_agent.create_knowledge_base(
    name=KB_NAME,
    description="Knowledge Base created with S3 as data source and OpenSearch as vector store.",
    roleArn=BEDROCK_ROLE_ARN,
    knowledgeBaseConfiguration={
        "type": "VECTOR",
        "vectorKnowledgeBaseConfiguration": {
            "embeddingModelArn": EMBEDDING_MODEL_ARN
        }
    },
    storageConfiguration={
        "type": "OPENSEARCH_SERVERLESS",
        "opensearchServerlessConfiguration": {
            "collectionArn": OPENSEARCH_COLLECTION_ARN,
            "vectorIndexName": OPENSEARCH_VECTOR_INDEX_NAME,
            "fieldMapping": {
                "vectorField": "vectors",
                "textField": "text",
                "metadataField": "metadata"
            }
        }
    }
)

knowledge_base_id = create_kb_response['knowledgeBase']['knowledgeBaseId']
print(f"Knowledge Base Created Successfully! ID: {knowledge_base_id}")

In [ ]:
response = bedrock_agent.list_knowledge_bases()
response